<a href="https://colab.research.google.com/github/Belal-AI/Data-Science-And-Ml-projects/blob/main/Copy_of_rewardmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
!pip install trl

In [ ]:
import random
import pandas as pd
from operator import itemgetter
import torch
import warnings
warnings.filterwarnings('ignore')
from datasets import Dataset, load_dataset
from transformers import AutoModelForSequenceClassification,AutoTokenizer,TrainingArguments
from trl import RewardTrainer

In [ ]:
from datasets import load_dataset

ds = load_dataset("nvidia/HelpSteer")

In [ ]:
df = ds['train'].to_pandas()


In [ ]:
df['question']=df['prompt']
df['answer']=df['response']

In [ ]:
df.drop(columns=['prompt','response'],inplace=True)

In [ ]:
df['feedback']=df['helpfulness']
df.drop(columns=['helpfulness'],inplace=True)

In [ ]:
df.drop(columns=['correctness','coherence',	'complexity',	'verbosity'],inplace=True)

In [ ]:
df['tup']=list(zip(df['answer'],df['feedback']))

In [ ]:
df_g=df.groupby('question')['tup'].apply(list).reset_index()
df_g

,question,tup
0,A group of people have been wrongfully convict...,"[(My name is [Name], and I am one of the wrong..."
1,A scientist is working on a spectrograph to de...,[(As the scientist worked on their spectrograp...
2,A young woman is shocked to discover that her ...,"[(The young woman, who is unnamed, is taken ab..."
3,Act as a fraudster who has just been arrested ...,[(Subject: My sincere apologies for my recent ...
4,Act as if you are a Buddhist monk who just fou...,"[(To whom it may concern,\n\nI am a Buddhist m..."
...,...,...
9939,given this background information delimited in...,[(The cph group's focus on biodegradable adhes...
9940,given this background information delimited in...,[(Slide 1: Introduction to Naval Gunnery\n- De...
9941,given this background information delimited in...,[(1999: House no food and he show up on the th...
9942,given this background information delimited in...,[([Your wounds]: [Scabs]: [1955]\n[Your kingdo...


In [ ]:


df_g['tup']=df_g['tup'].apply(lambda x : sorted(x,key=itemgetter(1)) )
df_g['chosen']=df_g['tup'].apply(lambda x:x[-1][0])
df_g['chosen_score']=df_g['tup'].apply(lambda x:x[-1][1])

df_g['rejected']=df_g['tup'].apply(lambda x:x[0][0])
df_g['rejectex_score']=df_g['tup'].apply(lambda x:x[0][1])
df_g

,question,tup,chosen,chosen_score,rejected,rejectex_score
0,A group of people have been wrongfully convict...,"[(Hi there, I am an AI assistant that can answ...","You are right, this should be a simple task fo...",3,"Hi there, I am an AI assistant that can answer...",0
1,A scientist is working on a spectrograph to de...,"[(This is an interesting topic, and I'd be hap...",The scientist was working on a spectrograph to...,3,"This is an interesting topic, and I'd be happy...",1
2,A young woman is shocked to discover that her ...,"[(The young woman, who is unnamed, is taken ab...","The young woman, who is unnamed, is taken abac...",3,"The young woman, who is unnamed, is taken abac...",3
3,Act as a fraudster who has just been arrested ...,[(I apologize for the deception. I just wanted...,"My dear victims,\n\nI am writing this letter t...",3,I apologize for the deception. I just wanted t...,2
4,Act as if you are a Buddhist monk who just fou...,"[(To whom it may concern,\n\nI am writing to i...",Here is a sample response for the prompt:\n\nD...,4,"To whom it may concern,\n\nI am writing to inf...",2
...,...,...,...,...,...,...
9939,given this background information delimited in...,[(Biodegradable adhesives produced by the cph ...,The cph group's focus on biodegradable adhesiv...,4,Biodegradable adhesives produced by the cph gr...,2
9940,given this background information delimited in...,[(- What is a gun?\n- Parts of a gun\n- Types ...,Slide 1: Introduction to Naval Gunnery\n- Defi...,4,- What is a gun?\n- Parts of a gun\n- Types of...,2
9941,given this background information delimited in...,[(1999: House no food and he show up on the th...,[2019]: [something there is that loves an arro...,3,1999: House no food and he show up on the thir...,1
9942,given this background information delimited in...,[([Your wounds]: [Scabs]: [1955]\n[Your kingdo...,Kaleidoscopticon: A kaleidoscope is a toy that...,2,[Your wounds]: [Scabs]: [1955]\n[Your kingdom]...,2


In [ ]:
df_g=df_g[ ( df_g['chosen_score']>=4.0 ) & (df_g['rejectex_score']<=3.0) ]
df_g

,question,tup,chosen,chosen_score,rejected,rejectex_score
4,Act as if you are a Buddhist monk who just fou...,"[(To whom it may concern,\n\nI am writing to i...",Here is a sample response for the prompt:\n\nD...,4,"To whom it may concern,\n\nI am writing to inf...",2
7,"Act as if you are a computer, and generate a c...",[(Sure! I'll be happy to help. Can you please ...,"User: Hi, I'm having trouble with your program...",4,Sure! I'll be happy to help. Can you please pr...,0
9,Act as if you are a head of lettuce about to b...,[(My journey started as a small seed in a gard...,"As a head of lettuce, I am excited to be chose...",4,My journey started as a small seed in a garden...,2
12,Act as if you are a medical researcher who is ...,"[(Okay, I am now a medical researcher interest...",To test the effects of different medications o...,4,"Okay, I am now a medical researcher interested...",3
13,Act as if you are a person with depression and...,"[(As someone who has experienced depression, I...",My depression feels like a dark cloud hanging ...,4,"As someone who has experienced depression, I c...",3
...,...,...,...,...,...,...
9931,given this background information delimited in...,"[(In the poem ""Three Day Road,"" the speaker di...","The poem ""You"" by Natalie Diaz describes a con...",4,"In the poem ""Three Day Road,"" the speaker disc...",3
9934,given this background information delimited in...,[(The imagery in the poem creates a sense of i...,The imagery in the poem is a powerful tool tha...,4,The imagery in the poem creates a sense of int...,3
9938,given this background information delimited in...,"[(fall in love, 2), (Immortally fall in love, ...",The author feels passionately about the other ...,4,fall in love,2
9939,given this background information delimited in...,[(Biodegradable adhesives produced by the cph ...,The cph group's focus on biodegradable adhesiv...,4,Biodegradable adhesives produced by the cph gr...,2


In [ ]:
rows = []
for record in df_g.itertuples(index=True, name='Pandas'):
    if record is None or len(record) == 0:
        continue
    rows.append({
        "instruction": record.question,
        "chosen_response": record.chosen,
        "rejected_response": record.rejected
    })
prepared_dataset = Dataset.from_list(rows)
prepared_dataset.to_pandas()

,instruction,chosen_response,rejected_response
0,Act as if you are a Buddhist monk who just fou...,Here is a sample response for the prompt:\n\nD...,"To whom it may concern,\n\nI am writing to inf..."
1,"Act as if you are a computer, and generate a c...","User: Hi, I'm having trouble with your program...",Sure! I'll be happy to help. Can you please pr...
2,Act as if you are a head of lettuce about to b...,"As a head of lettuce, I am excited to be chose...",My journey started as a small seed in a garden...
3,Act as if you are a medical researcher who is ...,To test the effects of different medications o...,"Okay, I am now a medical researcher interested..."
4,Act as if you are a person with depression and...,My depression feels like a dark cloud hanging ...,"As someone who has experienced depression, I c..."
...,...,...,...
4490,given this background information delimited in...,"The poem ""You"" by Natalie Diaz describes a con...","In the poem ""Three Day Road,"" the speaker disc..."
4491,given this background information delimited in...,The imagery in the poem is a powerful tool tha...,The imagery in the poem creates a sense of int...
4492,given this background information delimited in...,The author feels passionately about the other ...,fall in love
4493,given this background information delimited in...,The cph group's focus on biodegradable adhesiv...,Biodegradable adhesives produced by the cph gr...


In [ ]:
model_name = "distilroberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": 512, "return_tensors": "pt"}
    prompt_plus_chosen_response = examples["instruction"] + "\n" + examples["chosen_response"]
    prompt_plus_rejected_response = examples["instruction"] + "\n" + examples["rejected_response"]

    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)
    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }


In [ ]:
prepared_dataset=prepared_dataset.map(formatting_func)

Map:   0%|          | 0/4495 [00:00<?, ? examples/s]

In [ ]:
prepared_dataset.to_pandas()

,instruction,chosen_response,rejected_response,input_ids_chosen,attention_mask_chosen,input_ids_rejected,attention_mask_rejected
0,Act as if you are a Buddhist monk who just fou...,Here is a sample response for the prompt:\n\nD...,"To whom it may concern,\n\nI am writing to inf...","[0, 26880, 25, 114, 47, 32, 10, 21345, 35949, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 26880, 25, 114, 47, 32, 10, 21345, 35949, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"Act as if you are a computer, and generate a c...","User: Hi, I'm having trouble with your program...",Sure! I'll be happy to help. Can you please pr...,"[0, 26880, 25, 114, 47, 32, 10, 3034, 6, 8, 53...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 26880, 25, 114, 47, 32, 10, 3034, 6, 8, 53...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Act as if you are a head of lettuce about to b...,"As a head of lettuce, I am excited to be chose...",My journey started as a small seed in a garden...,"[0, 26880, 25, 114, 47, 32, 10, 471, 9, 24515,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 26880, 25, 114, 47, 32, 10, 471, 9, 24515,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Act as if you are a medical researcher who is ...,To test the effects of different medications o...,"Okay, I am now a medical researcher interested...","[0, 26880, 25, 114, 47, 32, 10, 1131, 9338, 54...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 26880, 25, 114, 47, 32, 10, 1131, 9338, 54...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Act as if you are a person with depression and...,My depression feels like a dark cloud hanging ...,"As someone who has experienced depression, I c...","[0, 26880, 25, 114, 47, 32, 10, 621, 19, 6943,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 26880, 25, 114, 47, 32, 10, 621, 19, 6943,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...,...
4490,given this background information delimited in...,"The poem ""You"" by Natalie Diaz describes a con...","In the poem ""Three Day Road,"" the speaker disc...","[0, 31699, 42, 3618, 335, 45382, 4560, 11, 494...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 31699, 42, 3618, 335, 45382, 4560, 11, 494...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4491,given this background information delimited in...,The imagery in the poem is a powerful tool tha...,The imagery in the poem creates a sense of int...,"[0, 31699, 42, 3618, 335, 45382, 4560, 11, 494...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 31699, 42, 3618, 335, 45382, 4560, 11, 494...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4492,given this background information delimited in...,The author feels passionately about the other ...,fall in love,"[0, 31699, 42, 3618, 335, 45382, 4560, 11, 494...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 31699, 42, 3618, 335, 45382, 4560, 11, 494...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4493,given this background information delimited in...,The cph group's focus on biodegradable adhesiv...,Biodegradable adhesives produced by the cph gr...,"[0, 31699, 42, 3618, 335, 45382, 4560, 11, 494...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 31699, 42, 3618, 335, 45382, 4560, 11, 494...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
training_args = TrainingArguments(
    output_dir="/reward_model",
    per_device_train_batch_size=16,
    evaluation_strategy="steps",
    logging_steps=1,
    num_train_epochs = 10,
    report_to=None,
)
# Loading the RewardTrainer from TRL
trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=prepared_dataset,
)
trainer.train()

In [ ]:
!pip install langchain_contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.2/420.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.6 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.35
    Uninstalling SQLAlchemy-2.0.35:
      Successfully uninstalled SQLAlchemy-2.0.35
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.9.2
    Uninstalling pydantic-2.9.2:
      Successfully uninstalled pydantic-2.9.2
  Attempting uninstall: dataclasses-json
    Found existing installation: dataclasses-json 0.6.7
    Uninstalling dataclasses-json-0.6.7:
      Successfully uninstalled dataclasses-json-0.6.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.1
    Uninstalling langchain-0.3.1:
      Successfully uninstalled langchain-0.3

In [ ]:
from langchain.llms import FakeLLM
fake_llm = FakeLLM(responses=["Hello"])

ImportError: cannot import name 'FakeLLM' from 'langchain.llms' (/usr/local/lib/python3.10/dist-packages/langchain/llms/__init__.py)

In [ ]:
import langchain


In [ ]:
!pip show langchain


Name: langchain
Version: 0.3.1
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community


In [ ]:
from langchain.llms import FakeLLM

ImportError: cannot import name 'FakeLLM' from 'langchain.llms' (/usr/local/lib/python3.10/dist-packages/langchain/llms/__init__.py)

In [ ]:
from langchain.llms.fake import FakeListLLM
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

ModuleNotFoundError: No module named 'pydantic.deprecated.config'